In [ ]:
## Se deben instalar todas las librerias que se colocan en el cajon de abajo

In [ ]:
!pip install pandas   #biblioteca de Python utilizada para el análisis y la manipulación de datos.
!pip install numpy    # biblioteca de Python utilizada para el cálculo numérico y matemático.
!pip install pandas_profiling
!pip install --upgrade google-cloud-bigquery
!pip install pandas-gbq

In [ ]:
from google.colab import drive    #Estas líneas de código se utilizan para montar el sistema de archivos de Google Drive en Google Colab.
drive.mount('/content/drive')
import pandas as pd   
import numpy as np  
from google.cloud import bigquery
from google.oauth2 import service_account
import random
import string
import re


Mounted at /content/drive



 ## Credenciales:



In [ ]:
## El archivo Json que nos arroja Bigquery con credenciales, debera ser colocado en una carpeta dentro de nuestros archivos de drive y de alli se llamara invocandolo desde la ruta donde se encuentra.

In [ ]:
credentials = service_account.Credentials.from_service_account_file("/content/drive/MyDrive/Gestion_de_Datos/Entrega_Proyecto/datos_clave.json", scopes=["https://www.googleapis.com/auth/cloud-platform"])
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [ ]:
## Se llaman los archivos originales y se hace los tratamientos correspondientes.
artists_mod = pd.read_csv("/content/drive/MyDrive/Gestion_de_Datos/Entrega_Proyecto/artists_mod.csv") #La función "pd.read_csv()" de la biblioteca Pandas toma como argumento la ruta del archivo CSV y devuelve un DataFrame de Pandas que contiene los datos del archivo. 
tracks_mod =  pd.read_csv("/content/drive/MyDrive/Gestion_de_Datos/Entrega_Proyecto/tracks_mod.csv")

# Desarrollo ETL

## artists_mod

In [ ]:
artists_mod.head(5)##Esta línea de código  la función "head()" del DataFrame "artists_mod" para mostrar las primeras 5 filas del DataFrame en la consola.

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0


In [ ]:
artists_mod.isna().sum() #Esta funcion  devuelve la suma de valores nulos (NaN) por columna en un DataFrame llamado artists_mod. 
artists_mod = artists_mod.dropna() # Quitamos los registros NA en el dataframe.
artists_mod = artists_mod.assign( genres = artists_mod["genres"].replace('[]','[Uknown]'))
artists_mod["genres"] = artists_mod["genres"].str.replace("'","")
artists_mod["genres"] = artists_mod["genres"].str.replace("[","")
artists_mod["genres"] = artists_mod["genres"].str.replace("]","")


<ipython-input-12-382ea0537390>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  artists_mod["genres"] = artists_mod["genres"].str.replace("[","")
<ipython-input-12-382ea0537390>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  artists_mod["genres"] = artists_mod["genres"].str.replace("]","")


In [ ]:
# Configuramos los campos que tendra la tabla en BQ.
job_config_artist = bigquery.LoadJobConfig(
    schema=[
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField("id", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("followers", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("genres", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("name", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("popularity", bigquery.enums.SqlTypeNames.FLOAT64)
    ],
    # Drod and re-create table, if exist
    write_disposition="WRITE_TRUNCATE"
)

In [ ]:
## Establecemos el nombre y la ruta de la tabla en bigquery.
BQ_TABLE_NAME_artist = f"tablas_proyecto.artists_mod"


In [ ]:
# Enviamos la Tabla a Bigquery.
job = client.load_table_from_dataframe(artists_mod, BQ_TABLE_NAME_artist, job_config=job_config_artist)
job.result()

LoadJob<project=proyecto-gestion-datos, location=US, id=8959e00d-59a4-4823-8b57-d93123db85af>

In [ ]:
# Verificamos si el proceso de subida de la tabla a nuestro espacio fue exitoso.
table = client.get_table(BQ_TABLE_NAME_artist)
print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), BQ_TABLE_NAME_artist)) 

Loaded 1162084 rows and 5 columns to tablas_proyecto.artists_mod


## tracks_mod

In [ ]:
tracks_mod = tracks_mod.dropna()
tracks_mod["artists"] = tracks_mod["artists"].str.replace("'","")
tracks_mod["artists"] = tracks_mod["artists"].str.replace("[","")
tracks_mod["artists"] = tracks_mod["artists"].str.replace("]","")
tracks_mod["id_artists"] = tracks_mod["id_artists"].str.replace("'","")
tracks_mod["id_artists"] = tracks_mod["id_artists"].str.replace("[","")
tracks_mod["id_artists"] = tracks_mod["id_artists"].str.replace("]","")


<ipython-input-17-4bef33e128de>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tracks_mod["artists"] = tracks_mod["artists"].str.replace("[","")
<ipython-input-17-4bef33e128de>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tracks_mod["artists"] = tracks_mod["artists"].str.replace("]","")
<ipython-input-17-4bef33e128de>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tracks_mod["id_artists"] = tracks_mod["id_artists"].str.replace("[","")
<ipython-input-17-4bef33e128de>:7: FutureWarning: The default value of regex wi

In [ ]:
tracks_mod.head(3)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6.0,126903,0,Uli,45tIt06XoI0Iio4LBEVpls,1922-02-22,0.645,0.445,0.0,-13.338,1.0,0.4510,0.674,0.7440,0.151,0.127,104.851,3.0
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0.0,98200,0,Fernando Pessoa,14jtPCOoNZwquk5wd9DxrY,1922-06-01,0.695,0.263,0.0,-22.136,1.0,0.9570,0.797,0.0000,0.148,0.655,102.009,1.0
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0.0,181640,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922-03-21,0.434,0.177,1.0,-21.180,1.0,0.0512,0.994,0.0218,0.212,0.457,130.418,5.0


In [ ]:
tracks_mod.columns

Index(['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists',
       'id_artists', 'release_date', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')

In [ ]:
# Configuramos los campos que tendra la tabla en BQ.

job_config_tracks = bigquery.LoadJobConfig(
    schema=[
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField("id", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("name", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("popularity", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("duration_ms", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("explicit", bigquery.enums.SqlTypeNames.FLOAT64)
        ,bigquery.SchemaField("artists", bigquery.enums.SqlTypeNames.STRING)
        ,bigquery.SchemaField("id_artists", bigquery.enums.SqlTypeNames.STRING)
        ,bigquery.SchemaField("release_date", bigquery.enums.SqlTypeNames.STRING)
        ,bigquery.SchemaField("danceability", bigquery.enums.SqlTypeNames.FLOAT64)
        ,bigquery.SchemaField("energy", bigquery.enums.SqlTypeNames.FLOAT64)
        ,bigquery.SchemaField("key", bigquery.enums.SqlTypeNames.FLOAT64)
        ,bigquery.SchemaField("loudness", bigquery.enums.SqlTypeNames.FLOAT64)
        ,bigquery.SchemaField("mode", bigquery.enums.SqlTypeNames.FLOAT64)
        ,bigquery.SchemaField("speechiness", bigquery.enums.SqlTypeNames.FLOAT64)
        ,bigquery.SchemaField("acousticness", bigquery.enums.SqlTypeNames.FLOAT64)
        ,bigquery.SchemaField("instrumentalness", bigquery.enums.SqlTypeNames.FLOAT64)
        ,bigquery.SchemaField("liveness", bigquery.enums.SqlTypeNames.FLOAT64)
        ,bigquery.SchemaField("valence", bigquery.enums.SqlTypeNames.FLOAT64)
        ,bigquery.SchemaField("tempo", bigquery.enums.SqlTypeNames.FLOAT64)
        ,bigquery.SchemaField("time_signature", bigquery.enums.SqlTypeNames.FLOAT64)
    ],
    # Drod and re-create table, if exist
    write_disposition="WRITE_TRUNCATE"
)

In [ ]:
## Establecemos el nombre y la ruta de la tabla en bigquery.
BQ_TABLE_NAME_tracks = f"tablas_proyecto.tracks"


In [ ]:
# Enviamos la Tabla a Bigquery.
job = client.load_table_from_dataframe(tracks_mod, BQ_TABLE_NAME_tracks, job_config=job_config_tracks)
job.result()

LoadJob<project=proyecto-gestion-datos, location=US, id=7a9958c4-fa18-4549-8239-0035eb92bb85>

In [ ]:
# Verificamos si el proceso de subida de la tabla a nuestro espacio fue exitoso.
table = client.get_table(BQ_TABLE_NAME_tracks)
print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), BQ_TABLE_NAME_tracks)) 

Loaded 348066 rows and 20 columns to tablas_proyecto.tracks
